In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/world-cities/worldcities.csv
/kaggle/input/linkedin-job-posts-insights-dataset/linkedin_job_posts_insights.xlsx


### I'll be including a dataset containing information about countries which will help me deal with the utter messiness of the location column in our LinkedIn dataset. It is licensed under the Creative Commons Attribution 4.0 license. For further details regarding the license terms, [please refer to the source](https://simplemaps.com/data/world-cities).

In [2]:
df = pd.read_excel("/kaggle/input/linkedin-job-posts-insights-dataset/linkedin_job_posts_insights.xlsx")
world = pd.read_csv("/kaggle/input/world-cities/worldcities.csv")

# Data Cleaning

In [3]:
df.head()

,job_title,company_name,location,hiring_status,date,seniority_level,job_function,employment_type,industry
0,Store Business Manager - DAVID JONES CHERMSIDE,M.J. Bale,"Brisbane, Queensland, Australia",Be an early applicant,2023-04-13,Not Applicable,Sales and Business Development,Full-time,Government Administration
1,Full-time,Gatesman,"Chicago, IL",Be an early applicant,2023-03-31,NaN,NaN,NaN,NaN
2,Senior Machine Learning Engineer,Redwolf + Rosch,"Adelaide, South Australia, Australia",Be an early applicant,2023-04-25,Mid-Senior level,Engineering and Information Technology,Part-time,Internet Publishing
3,Senior Data Scientist,Bupa,"Melbourne, Victoria, Australia",Be an early applicant,2023-04-29,Entry level,Engineering and Information Technology,Full-time,"Technology, Information and Internet"
4,\n \n Solution Architect\n ...,\n Xybion Digital\n,"\n Chennai, Tamil Nadu, India\n ...",\n Be an early applicant\n,2023-01-26,\n Mid-Senior level\n,\n Engineering and Information Tech...,\n Full-time\n,"\n IT Services and IT Consulting, Sof..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31597 entries, 0 to 31596
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   job_title        31571 non-null  object        
 1   company_name     30657 non-null  object        
 2   location         31588 non-null  object        
 3   hiring_status    31597 non-null  object        
 4   date             31597 non-null  datetime64[ns]
 5   seniority_level  30289 non-null  object        
 6   job_function     30007 non-null  object        
 7   employment_type  30006 non-null  object        
 8   industry         29586 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 2.2+ MB


#### I'll just get rid of all nulls. There aren't that many and due to the (mostly) nominal nature of the data, I think it's ok to do that ~~and make this less painful for me~~.

In [5]:
df.dropna(inplace=True)
df = df.reset_index()

In [6]:
df["seniority_level"].unique()

array(['Not Applicable', 'Mid-Senior level', 'Entry level',
       '\n            Mid-Senior level\n          ', 'Associate',
       '\n            Not Applicable\n          ',
       '\n            Associate\n          ',
       '\n            Entry level\n          ',
       '\n            Director\n          ', 'Executive', 'Director',
       '\n            Internship\n          ',
       '\n            Executive\n          ', 'Internship',
       '\n            Senior level\n          '], dtype=object)

In [7]:
df["employment_type"].unique()

array(['Full-time', 'Part-time', '\n          Full-time\n        ',
       '\n        \n          Employment type\n        \n        \n          Full-time\n        \n      ',
       'Contract', '\n          Contract\n        ',
       '\n        \n          Employment type\n        \n        \n          Part-time\n        \n      ',
       '\n        \n          Employment type\n        \n        \n          Internship\n        \n      ',
       '\n          Part-time\n        ', 'Internship',
       '\n          Internship\n        ', 'Volunteer', 'Temporary',
       'Other', '\n          Temporary\n        ',
       '\n          Other\n        ',
       '\n        \n          Employment type\n        \n        \n          Contract\n        \n      ',
       '\n        \n          Employment type\n        \n        \n          Other\n        \n      '],
      dtype=object)

#### We'll have to strip the unnecessary characters "Employment type\n", "\n" and the spaces on the sides from the seniority_level and employment_type columns. In fact, let's make sure we remove these characters from all columns. 

In [8]:
def line_break_cleaner(word):
    return word.strip("\n ")

def employment_type_cleaner(employment_type):
    return employment_type.lstrip("Employment type\n")

for df_column in df.columns:
    try:
        df[df_column] = df[df_column].apply(func=line_break_cleaner)
    except AttributeError:
        continue

df["employment_type"] = df["employment_type"].apply(employment_type_cleaner)

In [9]:
print(df["seniority_level"].unique())
print(df["employment_type"].unique())

['Not Applicable' 'Mid-Senior level' 'Entry level' 'Associate' 'Director'
 'Executive' 'Internship' 'Senior level']
['Full-time' 'Part-time' 'Contract' 'Internship' 'Volunteer' 'Temporary'
 'Other']


In [10]:
df[df.duplicated() == True]

,index,job_title,company_name,location,hiring_status,date,seniority_level,job_function,employment_type,industry


#### After checking the job_title column for a while I realized so many of the strings had nonsensical characters at both sides starting with numbers, hashes, dashes with no followups, unclosed (useless) quotations and so on. Not sure if I missed more but at least I can strip these.

In [11]:
def job_title_cleaner(job):
    return job.strip('#1234567890 -"')

df["job_title"] = df["job_title"].apply(func=job_title_cleaner)
df = df.drop(columns="index")

#### Whilst inspecting the location column, US states have been abbreviated by ISO 3166-2 codes, and (as far as I searched) I did not find any other states from other countries abbreviated the same way. So I just added "United States" to any string in the column with such codes.

In [12]:
def united_states(sentence):
    return sentence.__add__(", United States") if sentence[-2:] == sentence[-2:].upper() else sentence

df["location"] = df["location"].apply(func=united_states)

In [13]:
df.head(2)

,job_title,company_name,location,hiring_status,date,seniority_level,job_function,employment_type,industry
0,Store Business Manager - DAVID JONES CHERMSIDE,M.J. Bale,"Brisbane, Queensland, Australia",Be an early applicant,2023-04-13,Not Applicable,Sales and Business Development,Full-time,Government Administration
1,Senior Machine Learning Engineer,Redwolf + Rosch,"Adelaide, South Australia, Australia",Be an early applicant,2023-04-25,Mid-Senior level,Engineering and Information Technology,Part-time,Internet Publishing


### Now let's check the world dataframe.

In [14]:
world.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0


#### The city column contains city names, some of which in their actual non-English form or not exactly using English alphabet, whereas the city_ascii includes the city name in English so I'll remove the city column and rename the city_ascii column into city. I'll also remove all other columns except for country and iso3 (which I'll rename to iso_alpha3). The reason will become clear when plotting.

In [15]:
world = world.drop(columns=["city", "iso2", "admin_name", "capital", "lat", "lng", "population"]).rename(columns={'city_ascii':'city', 'iso3':'iso_alpha3'})

for column in world.columns:
    try:
        world[column].apply(func=line_break_cleaner)
    except AttributeError:
        continue

In [16]:
world.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44691 entries, 0 to 44690
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   city        44691 non-null  object
 1   country     44691 non-null  object
 2   iso_alpha3  44691 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


In [17]:
world.head()

,city,country,iso_alpha3
0,Tokyo,Japan,JPN
1,Jakarta,Indonesia,IDN
2,Delhi,India,IND
3,Guangzhou,China,CHN
4,Mumbai,India,IND


#### I think this should be good enough.

# Data Wrangling and Visualization

#### Let's make plots to find:

##### 1. Most popular job titles worldwide. 
##### 2. The companies with the most job postings.
##### 3. Industries with the highest job counts.

#### I chose to show the to 30 in each category to ensure the plots aren't jumbled, and 30 is more than enough when trying to identify top rankings.

For industries, one of the industries that appeared the most is "IT Services and IT Consulting, Software Development, and Technology, Information and Internet", but I omitted it since each feature already exists as a separate industry tag and they happen to appear in our top 30 list.

In [18]:
df_popular_jobs = df.groupby("job_title").size().reset_index(name='count').sort_values(by="count", ascending=False)[:30]
fig = px.bar(df_popular_jobs, x="job_title", y="count", title="The 30 Most popular job titles")
fig.update_layout(xaxis=dict(automargin=True, tickangle=45),
                  margin=dict(l=50, r=50, t=50, b=50),
                  hovermode='x')
fig.show()
print("____________________________________________________________")

df_companies_with_highest_job_postings = df.groupby("company_name").size().reset_index(name="count").sort_values(by="count", ascending=False)[:30]
fig_2 = px.bar(df_companies_with_highest_job_postings, x="company_name", y="count", title="The 30 companies with the greatest job postings")

fig_2.update_layout(xaxis=dict(automargin=True, tickangle=45),
                 margin=dict(l=50, r=50, t=50, b=50),
                 hovermode='x')
fig_2.show()
print("____________________________________________________________")

df_industries_with_highest_job_postings = df.groupby("industry").size().reset_index(name="count").sort_values(by="count", ascending=False)[:31]
df_industries_with_highest_job_postings = df_industries_with_highest_job_postings[df_industries_with_highest_job_postings["industry"] != "IT Services and IT Consulting, Software Development, and Technology, Information and Internet"]
fig_3 = px.bar(df_industries_with_highest_job_postings, x="industry", y="count", title="The 30 industries with the greatest job postings")
fig_3.update_layout(xaxis=dict(automargin=True, tickangle=45),
                  margin=dict(l=50, r=50, t=50, b=50),
                  hovermode='x')
fig_3.show()

____________________________________________________________


____________________________________________________________


#### Now that we found the most popular jobs, companies and industries, I'll make plots for these top listings showcasing their changes through time.

In [19]:
df_evolution_popular_jobs = df.groupby(["job_title", "date"]).size().reset_index(name="count")
evolution_of_popular_jobs_count = dict()

for i in range(30):
    evolution_of_popular_jobs_count[i] = df_evolution_popular_jobs[df_evolution_popular_jobs["job_title"] == df_popular_jobs.iloc[i]["job_title"]]

traces_1 = dict()
for i in range(30):
    traces_1[i] = px.line(evolution_of_popular_jobs_count[i], x="date", y="count", markers=True)
    

fig = make_subplots(rows=1, cols=1)

for i in range(30):
    fig.add_trace(traces_1[i].data[0])

for i in range(1, 30):
    fig.data[i].visible = False

fig.update_layout(
    title="Available Senior Data Scientist jobs varying with time",
    xaxis_title="Date",
    yaxis_title="Count",    
    updatemenus=[dict(buttons=list([dict(label=f"{df_popular_jobs.iloc[i]['job_title']}", method="update", 
                                         args=[{"visible": [(j == i) for j in range(30)]}, {"title": f"Available {df_popular_jobs.iloc[i]['job_title']} jobs varying with time"}])
                                    for i in range(30)]), direction="down", showactive=True, x=1, xanchor="left", y=1, yanchor="top"),])

fig.show()
print("_________________________________________________________________________________________________")

df_evolution_popular_companies = df.groupby(["company_name", "date"]).size().reset_index(name="count").sort_values(by="date", ascending=True)
evolution_of_popular_companies_count = dict()

for i in range(30):
    evolution_of_popular_companies_count[i] = df_evolution_popular_companies[df_evolution_popular_companies["company_name"] == df_companies_with_highest_job_postings.iloc[i]["company_name"]]

traces_2 = dict()
for i in range(30):
    traces_2[i] = px.line(evolution_of_popular_companies_count[i], x="date", y="count", markers=True)
    

fig_2 = make_subplots(rows=1, cols=1)

for i in range(30):
    fig_2.add_trace(traces_2[i].data[0])

for i in range(1, 30):
    fig_2.data[i].visible = False

fig_2.update_layout(
    title = "Recruiting from Scratch's job postings varying with time",
    xaxis_title="Date",
    yaxis_title="Count",    
    updatemenus=[dict(buttons=list([dict(label=f"{df_companies_with_highest_job_postings.iloc[i]['company_name']}", method="update", 
                                         args=[{"visible": [(j == i) for j in range(30)]}, {"title": f"{df_companies_with_highest_job_postings.iloc[i]['company_name']}'s job postings varying with time"}])
                                    for i in range(30)]), direction="down", showactive=True, x=1, xanchor="left", y=1, yanchor="top"),])

fig_2.show()
print("_________________________________________________________________________________________________")

df_evolution_popular_industries = df.groupby(["industry", "date"]).size().reset_index(name="count").sort_values(by="date", ascending=True)
evolution_of_popular_industries_count = dict()

for i in range(30):
    evolution_of_popular_industries_count[i] = df_evolution_popular_industries[df_evolution_popular_industries["industry"] == df_industries_with_highest_job_postings.iloc[i]["industry"]]

traces_3 = dict()
for i in range(30):
    traces_3[i] = px.line(evolution_of_popular_industries_count[i], x="date", y="count", markers=True)
    

fig_3 = make_subplots(rows=1, cols=1)

for i in range(30):
    fig_3.add_trace(traces_3[i].data[0])

for i in range(1, 30):
    fig_3.data[i].visible = False

fig_3.update_layout(
    title = "Job postings in the Technology, Information and Internet industry varying with time",
    xaxis_title="Date",
    yaxis_title="Count",    
    updatemenus=[dict(buttons=list([dict(label=f"{df_industries_with_highest_job_postings.iloc[i]['industry']}", method="update", 
                                         args=[{"visible": [(j == i) for j in range(30)]}, {"title": f"Job postings in the {df_industries_with_highest_job_postings.iloc[i]['industry']} industry varying with time"}])
                                    for i in range(30)]), direction="down", showactive=True, x=1, xanchor="left", y=1, yanchor="top"),])

fig_3.show()

_________________________________________________________________________________________________


_________________________________________________________________________________________________


#### Great. Now let's look at the distribution of employment types across all jobs.

In [20]:
df_employment_type_counts = df.groupby("employment_type").size().reset_index(name="count").sort_values(by="count")
fig = px.pie(df_employment_type_counts, values="count", names="employment_type", title="Frequency of different employment types")
fig.show()

#### 

#### Now the greatest hassle: location. I'm just interested in identifying the country in which each job posting is from ~~totally because I'm not procrastinating~~. Using the world dataset, and a bunch of regex, I created a dictionary including dataframes, split per country, and each includes the number of jobs grouped by date.

In [21]:
world_countries = sorted(world["country"].unique().tolist())
jobs_by_country = dict()
for country in world_countries:
    pattern = "|".join([rf"\b{x}\b" for x in world["city"][world["country"] == country].unique()])
    pattern_we_dont_want = "|".join([rf"\b{x}\b" for x in world_countries if x != country])
    df_country = df[df["location"].str.match(pattern)][~df["location"].str.contains(pattern_we_dont_want, regex=True)]
    df_country = df_country.groupby("date").size().reset_index(name="count").sort_values(by="date",ascending=True)
    df_country["country"] = country
    jobs_by_country[country] = df_country

relevant_countries = [country for country in world_countries if not jobs_by_country[country].empty]

#### Now for every country that had some jobs posted on LinkedIn (omitting those that aren't amongst the dataset), I plotted the counts against the date to see how counts vary in time.

In [22]:
traces = dict()
L = len(relevant_countries)
for i in range(L):
    sub_fig = px.bar(jobs_by_country[relevant_countries[i]], x="date", y="count")
    sub_fig.update_layout(xaxis=dict(automargin=True, tickangle=25),
                margin=dict(l=50, r=50, t=50, b=50),
                hovermode='x')
    traces[i] = sub_fig

fig = make_subplots(rows=1, cols=1)

for i in range(L):
    fig.add_trace(traces[i].data[0])

for i in range(1, L):
    fig.data[i].visible = False

fig.update_layout(
    title = "Job distribution varying with time in Argentina",
    xaxis_title="Date",
    yaxis_title="Count",    
    updatemenus=[dict(buttons=list([dict(label=f"{relevant_countries[i]}", method="update", 
                                         args=[{"visible": [(j == i) for j in range(L)]}, {"title": f"Job distribution varying with time in {relevant_countries[i]}"}])
                                    for i in range(L)]), direction="down", showactive=True, x=1, xanchor="left", y=1, yanchor="top"),])

fig.show()

#### Here's a little geoplot showcasing the average number of job postings per date per country. This is where I make use of the ISO 3166-1 alpha-3 codes which I kept in the world dataframe.

To plot the geoplot, I needed to identify the continents in which each country is from, so I used a dataframe including countries and the corresponding continent.

In [23]:
country_continent = pd.DataFrame([
            {'timezones': ['Aisa/China'], 'code': 'CN', 'continent': 'Asia', 'name': 'China', 'capital': 'Beijing'},
            {'timezones': ['Europe/Netherlands'], 'code': 'NL', 'continent': 'Europe', 'name': 'Netherlands', 'capital': 'Amsterdam'},
            {'timezones': ['Europe/Czechia'], 'code': 'CZ', 'continent': 'Europe', 'name': 'Czechia', 'capital': 'Prague'},
            {'timezones': ['Europe/Ireland'], 'code': 'IE', 'continent': 'Europe', 'name': 'Ireland', 'capital': 'Dublin'},
            {'timezones': ['Europe/Gibraltar'], 'code': 'GI', 'continent': 'Europe', 'name': 'Gibraltar', 'capital': 'Gibraltar City'},
            {'timezones': ['Europe/Andorra'], 'code': 'AD', 'continent': 'Europe', 'name': 'Andorra', 'capital': 'Andorra la Vella'},
            {'timezones': ['Asia/Kabul'], 'code': 'AF', 'continent': 'Asia', 'name': 'Afghanistan', 'capital': 'Kabul'},
            {'timezones': ['America/Antigua'], 'code': 'AG', 'continent': 'North America', 'name': 'Antigua and Barbuda', 'capital': "St. John's"},
            {'timezones': ['Europe/Tirane'], 'code': 'AL', 'continent': 'Europe', 'name': 'Albania', 'capital': 'Tirana'},
            {'timezones': ['Asia/Yerevan'], 'code': 'AM', 'continent': 'Asia', 'name': 'Armenia', 'capital': 'Yerevan'},
            {'timezones': ['Africa/Luanda'], 'code': 'AO', 'continent': 'Africa', 'name': 'Angola', 'capital': 'Luanda'},
            {'timezones': ['America/Argentina/Buenos_Aires', 'America/Argentina/Cordoba', 'America/Argentina/Jujuy', 'America/Argentina/Tucuman', 'America/Argentina/Catamarca', 'America/Argentina/La_Rioja', 'America/Argentina/San_Juan', 'America/Argentina/Mendoza', 'America/Argentina/Rio_Gallegos', 'America/Argentina/Ushuaia'], 'code': 'AR', 'continent': 'South America', 'name': 'Argentina', 'capital': 'Buenos Aires'},
            {'timezones': ['Europe/Vienna'], 'code': 'AT', 'continent': 'Europe', 'name': 'Austria', 'capital': 'Vienna'},
            {'timezones': ['Australia/Lord_Howe', 'Australia/Hobart', 'Australia/Currie', 'Australia/Melbourne', 'Australia/Sydney', 'Australia/Broken_Hill', 'Australia/Brisbane', 'Australia/Lindeman', 'Australia/Adelaide', 'Australia/Darwin', 'Australia/Perth'], 'code': 'AU', 'continent': 'Oceania', 'name': 'Australia', 'capital': 'Canberra'},
            {'timezones': ['Asia/Baku'], 'code': 'AZ', 'continent': 'Asia', 'name': 'Azerbaijan', 'capital': 'Baku'},
            {'timezones': ['America/Barbados'], 'code': 'BB', 'continent': 'North America', 'name': 'Barbados', 'capital': 'Bridgetown'},
            {'timezones': ['Asia/Dhaka'], 'code': 'BD', 'continent': 'Asia', 'name': 'Bangladesh', 'capital': 'Dhaka'},
            {'timezones': ['Europe/Brussels'], 'code': 'BE', 'continent': 'Europe', 'name': 'Belgium', 'capital': 'Brussels'},
            {'timezones': ['Africa/Ouagadougou'], 'code': 'BF', 'continent': 'Africa', 'name': 'Burkina Faso', 'capital': 'Ouagadougou'},
            {'timezones': ['Europe/Sofia'], 'code': 'BG', 'continent': 'Europe', 'name': 'Bulgaria', 'capital': 'Sofia'},
            {'timezones': ['Asia/Bahrain'], 'code': 'BH', 'continent': 'Asia', 'name': 'Bahrain', 'capital': 'Manama'},
            {'timezones': ['Africa/Bujumbura'], 'code': 'BI', 'continent': 'Africa', 'name': 'Burundi', 'capital': 'Bujumbura'},
            {'timezones': ['Africa/Porto-Novo'], 'code': 'BJ', 'continent': 'Africa', 'name': 'Benin', 'capital': 'Porto-Novo'},
            {'timezones': ['Asia/Brunei'], 'code': 'BN', 'continent': 'Asia', 'name': 'Brunei Darussalam', 'capital': 'Bandar Seri Begawan'},
            {'timezones': ['America/La_Paz'], 'code': 'BO', 'continent': 'South America', 'name': 'Bolivia', 'capital': 'Sucre'},
            {'timezones': ['America/Noronha', 'America/Belem', 'America/Fortaleza', 'America/Recife', 'America/Araguaina', 'America/Maceio', 'America/Bahia', 'America/Sao_Paulo', 'America/Campo_Grande', 'America/Cuiaba', 'America/Porto_Velho', 'America/Boa_Vista', 'America/Manaus', 'America/Eirunepe', 'America/Rio_Branco'], 'code': 'BR', 'continent': 'South America', 'name': 'Brazil', 'capital': 'Bras\xc3\xadlia'},
            {'timezones': ['America/Nassau'], 'code': 'BS', 'continent': 'North America', 'name': 'Bahamas', 'capital': 'Nassau'},
            {'timezones': ['Asia/Thimphu'], 'code': 'BT', 'continent': 'Asia', 'name': 'Bhutan', 'capital': 'Thimphu'},
            {'timezones': ['Africa/Gaborone'], 'code': 'BW', 'continent': 'Africa', 'name': 'Botswana', 'capital': 'Gaborone'},
            {'timezones': ['Europe/Minsk'], 'code': 'BY', 'continent': 'Europe', 'name': 'Belarus', 'capital': 'Minsk'},
            {'timezones': ['America/Belize'], 'code': 'BZ', 'continent': 'North America', 'name': 'Belize', 'capital': 'Belmopan'},
            {'timezones': ['America/St_Johns', 'America/Halifax', 'America/Glace_Bay', 'America/Moncton', 'America/Goose_Bay', 'America/Blanc-Sablon', 'America/Montreal', 'America/Toronto', 'America/Nipigon', 'America/Thunder_Bay', 'America/Pangnirtung', 'America/Iqaluit', 'America/Atikokan', 'America/Rankin_Inlet', 'America/Winnipeg', 'America/Rainy_River', 'America/Cambridge_Bay', 'America/Regina', 'America/Swift_Current', 'America/Edmonton', 'America/Yellowknife', 'America/Inuvik', 'America/Dawson_Creek', 'America/Vancouver', 'America/Whitehorse', 'America/Dawson'], 'code': 'CA', 'continent': 'North America', 'name': 'Canada', 'capital': 'Ottawa'},
            {'timezones': ['Africa/Kinshasa', 'Africa/Lubumbashi'], 'code': 'CD', 'continent': 'Africa', 'name': 'Democratic Republic of the Congo', 'capital': 'Kinshasa'},
            {'timezones': ['Africa/Brazzaville'], 'code': 'CG', 'continent': 'Africa', 'name': 'Republic of the Congo', 'capital': 'Brazzaville'},
            {'timezones': ['Africa/Abidjan'], 'code': 'CI', 'continent': 'Africa', 'name': "C\xc3\xb4te d'Ivoire", 'capital': 'Yamoussoukro'},
            {'timezones': ['America/Santiago', 'Pacific/Easter'], 'code': 'CL', 'continent': 'South America', 'name': 'Chile', 'capital': 'Santiago'},
            {'timezones': ['Africa/Douala'], 'code': 'CM', 'continent': 'Africa', 'name': 'Cameroon', 'capital': 'Yaound\xc3\xa9'},
            {'timezones': ['Asia/Shanghai', 'Asia/Harbin', 'Asia/Chongqing', 'Asia/Urumqi', 'Asia/Kashgar'], 'code': 'CN', 'continent': 'Asia', 'name': "People's Republic of China", 'capital': 'Beijing'},
            {'timezones': ['America/Bogota'], 'code': 'CO', 'continent': 'South America', 'name': 'Colombia', 'capital': 'Bogot\xc3\xa1'},
            {'timezones': ['America/Costa_Rica'], 'code': 'CR', 'continent': 'North America', 'name': 'Costa Rica', 'capital': 'San Jos\xc3\xa9'},
            {'timezones': ['America/Havana'], 'code': 'CU', 'continent': 'North America', 'name': 'Cuba', 'capital': 'Havana'},
            {'timezones': ['Atlantic/Cape_Verde'], 'code': 'CV', 'continent': 'Africa', 'name': 'Cape Verde', 'capital': 'Praia'},
            {'timezones': ['Asia/Nicosia'], 'code': 'CY', 'continent': 'Asia', 'name': 'Cyprus', 'capital': 'Nicosia'},
            {'timezones': ['Europe/Prague'], 'code': 'CZ', 'continent': 'Europe', 'name': 'Czech Republic', 'capital': 'Prague'},
            {'timezones': ['Europe/Berlin'], 'code': 'DE', 'continent': 'Europe', 'name': 'Germany', 'capital': 'Berlin'},
            {'timezones': ['Africa/Djibouti'], 'code': 'DJ', 'continent': 'Africa', 'name': 'Djibouti', 'capital': 'Djibouti City'},
            {'timezones': ['Europe/Copenhagen'], 'code': 'DK', 'continent': 'Europe', 'name': 'Denmark', 'capital': 'Copenhagen'},
            {'timezones': ['America/Dominica'], 'code': 'DM', 'continent': 'North America', 'name': 'Dominica', 'capital': 'Roseau'},
            {'timezones': ['America/Santo_Domingo'], 'code': 'DO', 'continent': 'North America', 'name': 'Dominican Republic', 'capital': 'Santo Domingo'},
            {'timezones': ['America/Guayaquil', 'Pacific/Galapagos'], 'code': 'EC', 'continent': 'South America', 'name': 'Ecuador', 'capital': 'Quito'},
            {'timezones': ['Europe/Tallinn'], 'code': 'EE', 'continent': 'Europe', 'name': 'Estonia', 'capital': 'Tallinn'},
            {'timezones': ['Africa/Cairo'], 'code': 'EG', 'continent': 'Africa', 'name': 'Egypt', 'capital': 'Cairo'},
            {'timezones': ['Africa/Asmera'], 'code': 'ER', 'continent': 'Africa', 'name': 'Eritrea', 'capital': 'Asmara'},
            {'timezones': ['Africa/Addis_Ababa'], 'code': 'ET', 'continent': 'Africa', 'name': 'Ethiopia', 'capital': 'Addis Ababa'},
            {'timezones': ['Europe/Helsinki'], 'code': 'FI', 'continent': 'Europe', 'name': 'Finland', 'capital': 'Helsinki'},
            {'timezones': ['Pacific/Fiji'], 'code': 'FJ', 'continent': 'Oceania', 'name': 'Fiji', 'capital': 'Suva'},
            {'timezones': ['Europe/Paris'], 'code': 'FR', 'continent': 'Europe', 'name': 'France', 'capital': 'Paris'},
            {'timezones': ['Africa/Libreville'], 'code': 'GA', 'continent': 'Africa', 'name': 'Gabon', 'capital': 'Libreville'},
            {'timezones': ['Asia/Tbilisi'], 'code': 'GE', 'continent': 'Asia', 'name': 'Georgia', 'capital': 'Tbilisi'},
            {'timezones': ['Africa/Accra'], 'code': 'GH', 'continent': 'Africa', 'name': 'Ghana', 'capital': 'Accra'},
            {'timezones': ['Africa/Banjul'], 'code': 'GM', 'continent': 'Africa', 'name': 'The Gambia', 'capital': 'Banjul'},
            {'timezones': ['Africa/Conakry'], 'code': 'GN', 'continent': 'Africa', 'name': 'Guinea', 'capital': 'Conakry'},
            {'timezones': ['Europe/Athens'], 'code': 'GR', 'continent': 'Europe', 'name': 'Greece', 'capital': 'Athens'},
            {'timezones': ['America/Guatemala'], 'code': 'GT', 'continent': 'North America', 'name': 'Guatemala', 'capital': 'Guatemala City'},
            {'timezones': ['America/Guatemala'], 'code': 'GT', 'continent': 'North America', 'name': 'Haiti', 'capital': 'Port-au-Prince'},
            {'timezones': ['Africa/Bissau'], 'code': 'GW', 'continent': 'Africa', 'name': 'Guinea-Bissau', 'capital': 'Bissau'},
            {'timezones': ['America/Guyana'], 'code': 'GY', 'continent': 'South America', 'name': 'Guyana', 'capital': 'Georgetown'},
            {'timezones': ['America/Tegucigalpa'], 'code': 'HN', 'continent': 'North America', 'name': 'Honduras', 'capital': 'Tegucigalpa'},
            {'timezones': ['Europe/Budapest'], 'code': 'HU', 'continent': 'Europe', 'name': 'Hungary', 'capital': 'Budapest'},
            {'timezones': ['Asia/Jakarta', 'Asia/Pontianak', 'Asia/Makassar', 'Asia/Jayapura'], 'code': 'ID', 'continent': 'Asia', 'name': 'Indonesia', 'capital': 'Jakarta'},
            {'timezones': ['Europe/Dublin'], 'code': 'IE', 'continent': 'Europe', 'name': 'Republic of Ireland', 'capital': 'Dublin'},
            {'timezones': ['Asia/Jerusalem'], 'code': 'IL', 'continent': 'Asia', 'name': 'Israel', 'capital': 'Jerusalem'},
            {'timezones': ['Asia/Calcutta'], 'code': 'IN', 'continent': 'Asia', 'name': 'India', 'capital': 'New Delhi'},
            {'timezones': ['Asia/Baghdad'], 'code': 'IQ', 'continent': 'Asia', 'name': 'Iraq', 'capital': 'Baghdad'},
            {'timezones': ['Asia/Tehran'], 'code': 'IR', 'continent': 'Asia', 'name': 'Iran', 'capital': 'Tehran'},
            {'timezones': ['Atlantic/Reykjavik'], 'code': 'IS', 'continent': 'Europe', 'name': 'Iceland', 'capital': 'Reykjav\xc3\xadk'},
            {'timezones': ['Europe/Rome'], 'code': 'IT', 'continent': 'Europe', 'name': 'Italy', 'capital': 'Rome'},
            {'timezones': ['America/Jamaica'], 'code': 'JM', 'continent': 'North America', 'name': 'Jamaica', 'capital': 'Kingston'},
            {'timezones': ['Asia/Amman'], 'code': 'JO', 'continent': 'Asia', 'name': 'Jordan', 'capital': 'Amman'},
            {'timezones': ['Asia/Tokyo'], 'code': 'JP', 'continent': 'Asia', 'name': 'Japan', 'capital': 'Tokyo'},
            {'timezones': ['Africa/Nairobi'], 'code': 'KE', 'continent': 'Africa', 'name': 'Kenya', 'capital': 'Nairobi'},
            {'timezones': ['Asia/Bishkek'], 'code': 'KG', 'continent': 'Asia', 'name': 'Kyrgyzstan', 'capital': 'Bishkek'},
            {'timezones': ['Pacific/Tarawa', 'Pacific/Enderbury', 'Pacific/Kiritimati'], 'code': 'KI', 'continent': 'Oceania', 'name': 'Kiribati', 'capital': 'Tarawa'},
            {'timezones': ['Asia/Pyongyang'], 'code': 'KP', 'continent': 'Asia', 'name': 'North Korea', 'capital': 'Pyongyang'},
            {'timezones': ['Asia/Seoul'], 'code': 'KR', 'continent': 'Asia', 'name': 'South Korea', 'capital': 'Seoul'},
            {'timezones': ['Asia/Kuwait'], 'code': 'KW', 'continent': 'Asia', 'name': 'Kuwait', 'capital': 'Kuwait City'},
            {'timezones': ['Asia/Beirut'], 'code': 'LB', 'continent': 'Asia', 'name': 'Lebanon', 'capital': 'Beirut'},
            {'timezones': ['Europe/Vaduz'], 'code': 'LI', 'continent': 'Europe', 'name': 'Liechtenstein', 'capital': 'Vaduz'},
            {'timezones': ['Africa/Monrovia'], 'code': 'LR', 'continent': 'Africa', 'name': 'Liberia', 'capital': 'Monrovia'},
            {'timezones': ['Africa/Maseru'], 'code': 'LS', 'continent': 'Africa', 'name': 'Lesotho', 'capital': 'Maseru'},
            {'timezones': ['Europe/Vilnius'], 'code': 'LT', 'continent': 'Europe', 'name': 'Lithuania', 'capital': 'Vilnius'},
            {'timezones': ['Europe/Luxembourg'], 'code': 'LU', 'continent': 'Europe', 'name': 'Luxembourg', 'capital': 'Luxembourg City'},
            {'timezones': ['Europe/Riga'], 'code': 'LV', 'continent': 'Europe', 'name': 'Latvia', 'capital': 'Riga'},
            {'timezones': ['Africa/Tripoli'], 'code': 'LY', 'continent': 'Africa', 'name': 'Libya', 'capital': 'Tripoli'},
            {'timezones': ['Indian/Antananarivo'], 'code': 'MG', 'continent': 'Africa', 'name': 'Madagascar', 'capital': 'Antananarivo'},
            {'timezones': ['Pacific/Majuro', 'Pacific/Kwajalein'], 'code': 'MH', 'continent': 'Oceania', 'name': 'Marshall Islands', 'capital': 'Majuro'},
            {'timezones': ['Europe/Skopje'], 'code': 'MK', 'continent': 'Europe', 'name': 'Macedonia', 'capital': 'Skopje'},
            {'timezones': ['Africa/Bamako'], 'code': 'ML', 'continent': 'Africa', 'name': 'Mali', 'capital': 'Bamako'},
            {'timezones': ['Asia/Rangoon'], 'code': 'MM', 'continent': 'Asia', 'name': 'Myanmar', 'capital': 'Naypyidaw'},
            {'timezones': ['Asia/Ulaanbaatar', 'Asia/Hovd', 'Asia/Choibalsan'], 'code': 'MN', 'continent': 'Asia', 'name': 'Mongolia', 'capital': 'Ulaanbaatar'},
            {'timezones': ['Africa/Nouakchott'], 'code': 'MR', 'continent': 'Africa', 'name': 'Mauritania', 'capital': 'Nouakchott'},
            {'timezones': ['Europe/Malta'], 'code': 'MT', 'continent': 'Europe', 'name': 'Malta', 'capital': 'Valletta'},
            {'timezones': ['Indian/Mauritius'], 'code': 'MU', 'continent': 'Africa', 'name': 'Mauritius', 'capital': 'Port Louis'},
            {'timezones': ['Indian/Maldives'], 'code': 'MV', 'continent': 'Asia', 'name': 'Maldives', 'capital': 'Mal\xc3\xa9'},
            {'timezones': ['Africa/Blantyre'], 'code': 'MW', 'continent': 'Africa', 'name': 'Malawi', 'capital': 'Lilongwe'},
            {'timezones': ['America/Mexico_City', 'America/Cancun', 'America/Merida', 'America/Monterrey', 'America/Mazatlan', 'America/Chihuahua', 'America/Hermosillo', 'America/Tijuana'], 'code': 'MX', 'continent': 'North America', 'name': 'Mexico', 'capital': 'Mexico City'},
            {'timezones': ['Asia/Kuala_Lumpur', 'Asia/Kuching'], 'code': 'MY', 'continent': 'Asia', 'name': 'Malaysia', 'capital': 'Kuala Lumpur'},
            {'timezones': ['Africa/Maputo'], 'code': 'MZ', 'continent': 'Africa', 'name': 'Mozambique', 'capital': 'Maputo'},
            {'timezones': ['Africa/Windhoek'], 'code': 'NA', 'continent': 'Africa', 'name': 'Namibia', 'capital': 'Windhoek'},
            {'timezones': ['Africa/Niamey'], 'code': 'NE', 'continent': 'Africa', 'name': 'Niger', 'capital': 'Niamey'},
            {'timezones': ['Africa/Lagos'], 'code': 'NG', 'continent': 'Africa', 'name': 'Nigeria', 'capital': 'Abuja'},
            {'timezones': ['America/Managua'], 'code': 'NI', 'continent': 'North America', 'name': 'Nicaragua', 'capital': 'Managua'},
            {'timezones': ['Europe/Amsterdam'], 'code': 'NL', 'continent': 'Europe', 'name': 'Kingdom of the Netherlands', 'capital': 'Amsterdam'},
            {'timezones': ['Europe/Oslo'], 'code': 'NO', 'continent': 'Europe', 'name': 'Norway', 'capital': 'Oslo'},
            {'timezones': ['Asia/Katmandu'], 'code': 'NP', 'continent': 'Asia', 'name': 'Nepal', 'capital': 'Kathmandu'},
            {'timezones': ['Pacific/Nauru'], 'code': 'NR', 'continent': 'Oceania', 'name': 'Nauru', 'capital': 'Yaren'},
            {'timezones': ['Pacific/Auckland', 'Pacific/Chatham'], 'code': 'NZ', 'continent': 'Oceania', 'name': 'New Zealand', 'capital': 'Wellington'},
            {'timezones': ['Asia/Muscat'], 'code': 'OM', 'continent': 'Asia', 'name': 'Oman', 'capital': 'Muscat'},
            {'timezones': ['America/Panama'], 'code': 'PA', 'continent': 'North America', 'name': 'Panama', 'capital': 'Panama City'},
            {'timezones': ['America/Lima'], 'code': 'PE', 'continent': 'South America', 'name': 'Peru', 'capital': 'Lima'},
            {'timezones': ['Pacific/Port_Moresby'], 'code': 'PG', 'continent': 'Oceania', 'name': 'Papua New Guinea', 'capital': 'Port Moresby'},
            {'timezones': ['Asia/Manila'], 'code': 'PH', 'continent': 'Asia', 'name': 'Philippines', 'capital': 'Manila'},
            {'timezones': ['Asia/Karachi'], 'code': 'PK', 'continent': 'Asia', 'name': 'Pakistan', 'capital': 'Islamabad'},
            {'timezones': ['Europe/Warsaw'], 'code': 'PL', 'continent': 'Europe', 'name': 'Poland', 'capital': 'Warsaw'},
            {'timezones': ['Europe/Lisbon', 'Atlantic/Madeira', 'Atlantic/Azores'], 'code': 'PT', 'continent': 'Europe', 'name': 'Portugal', 'capital': 'Lisbon'},
            {'timezones': ['Pacific/Palau'], 'code': 'PW', 'continent': 'Oceania', 'name': 'Palau', 'capital': 'Ngerulmud'},
            {'timezones': ['America/Asuncion'], 'code': 'PY', 'continent': 'South America', 'name': 'Paraguay', 'capital': 'Asunci\xc3\xb3n'},
            {'timezones': ['Asia/Qatar'], 'code': 'QA', 'continent': 'Asia', 'name': 'Qatar', 'capital': 'Doha'},
            {'timezones': ['Europe/Bucharest'], 'code': 'RO', 'continent': 'Europe', 'name': 'Romania', 'capital': 'Bucharest'},
            {'timezones': ['Europe/Kaliningrad', 'Europe/Moscow', 'Europe/Volgograd', 'Europe/Samara', 'Asia/Yekaterinburg', 'Asia/Omsk', 'Asia/Novosibirsk', 'Asia/Krasnoyarsk', 'Asia/Irkutsk', 'Asia/Yakutsk', 'Asia/Vladivostok', 'Asia/Sakhalin', 'Asia/Magadan', 'Asia/Kamchatka', 'Asia/Anadyr'], 'code': 'RU', 'continent': 'Europe', 'name': 'Russia', 'capital': 'Moscow'},
            {'timezones': ['Africa/Kigali'], 'code': 'RW', 'continent': 'Africa', 'name': 'Rwanda', 'capital': 'Kigali'},
            {'timezones': ['Asia/Riyadh'], 'code': 'SA', 'continent': 'Asia', 'name': 'Saudi Arabia', 'capital': 'Riyadh'},
            {'timezones': ['Pacific/Guadalcanal'], 'code': 'SB', 'continent': 'Oceania', 'name': 'Solomon Islands', 'capital': 'Honiara'},
            {'timezones': ['Indian/Mahe'], 'code': 'SC', 'continent': 'Africa', 'name': 'Seychelles', 'capital': 'Victoria'},
            {'timezones': ['Africa/Khartoum'], 'code': 'SD', 'continent': 'Africa', 'name': 'Sudan', 'capital': 'Khartoum'},
            {'timezones': ['Europe/Stockholm'], 'code': 'SE', 'continent': 'Europe', 'name': 'Sweden', 'capital': 'Stockholm'},
            {'timezones': ['Asia/Singapore'], 'code': 'SG', 'continent': 'Asia', 'name': 'Singapore', 'capital': 'Singapore'},
            {'timezones': ['Europe/Ljubljana'], 'code': 'SI', 'continent': 'Europe', 'name': 'Slovenia', 'capital': 'Ljubljana'},
            {'timezones': ['Europe/Bratislava'], 'code': 'SK', 'continent': 'Europe', 'name': 'Slovakia', 'capital': 'Bratislava'},
            {'timezones': ['Africa/Freetown'], 'code': 'SL', 'continent': 'Africa', 'name': 'Sierra Leone', 'capital': 'Freetown'},
            {'timezones': ['Europe/San_Marino'], 'code': 'SM', 'continent': 'Europe', 'name': 'San Marino', 'capital': 'San Marino'},
            {'timezones': ['Africa/Dakar'], 'code': 'SN', 'continent': 'Africa', 'name': 'Senegal', 'capital': 'Dakar'},
            {'timezones': ['Africa/Mogadishu'], 'code': 'SO', 'continent': 'Africa', 'name': 'Somalia', 'capital': 'Mogadishu'},
            {'timezones': ['America/Paramaribo'], 'code': 'SR', 'continent': 'South America', 'name': 'Suriname', 'capital': 'Paramaribo'},
            {'timezones': ['Africa/Sao_Tome'], 'code': 'ST', 'continent': 'Africa', 'name': 'S\xc3\xa3o Tom\xc3\xa9 and Pr\xc3\xadncipe', 'capital': 'S\xc3\xa3o Tom\xc3\xa9'},
            {'timezones': ['Asia/Damascus'], 'code': 'SY', 'continent': 'Asia', 'name': 'Syria', 'capital': 'Damascus'},
            {'timezones': ['Africa/Lome'], 'code': 'TG', 'continent': 'Africa', 'name': 'Togo', 'capital': 'Lom\xc3\xa9'},
            {'timezones': ['Asia/Bangkok'], 'code': 'TH', 'continent': 'Asia', 'name': 'Thailand', 'capital': 'Bangkok'},
            {'timezones': ['Asia/Dushanbe'], 'code': 'TJ', 'continent': 'Asia', 'name': 'Tajikistan', 'capital': 'Dushanbe'},
            {'timezones': ['Asia/Ashgabat'], 'code': 'TM', 'continent': 'Asia', 'name': 'Turkmenistan', 'capital': 'Ashgabat'},
            {'timezones': ['Africa/Tunis'], 'code': 'TN', 'continent': 'Africa', 'name': 'Tunisia', 'capital': 'Tunis'},
            {'timezones': ['Pacific/Tongatapu'], 'code': 'TO', 'continent': 'Oceania', 'name': 'Tonga', 'capital': 'Nuku\xca\xbbalofa'},
            {'timezones': ['Europe/Istanbul'], 'code': 'TR', 'continent': 'Asia', 'name': 'Turkey', 'capital': 'Ankara'},
            {'timezones': ['America/Port_of_Spain'], 'code': 'TT', 'continent': 'North America', 'name': 'Trinidad and Tobago', 'capital': 'Port of Spain'},
            {'timezones': ['Pacific/Funafuti'], 'code': 'TV', 'continent': 'Oceania', 'name': 'Tuvalu', 'capital': 'Funafuti'},
            {'timezones': ['Africa/Dar_es_Salaam'], 'code': 'TZ', 'continent': 'Africa', 'name': 'Tanzania', 'capital': 'Dodoma'},
            {'timezones': ['Europe/Kiev', 'Europe/Uzhgorod', 'Europe/Zaporozhye', 'Europe/Simferopol'], 'code': 'UA', 'continent': 'Europe', 'name': 'Ukraine', 'capital': 'Kyiv'},
            {'timezones': ['Africa/Kampala'], 'code': 'UG', 'continent': 'Africa', 'name': 'Uganda', 'capital': 'Kampala'},
            {'timezones': ['America/New_York', 'America/Detroit', 'America/Kentucky/Louisville', 'America/Kentucky/Monticello', 'America/Indiana/Indianapolis', 'America/Indiana/Marengo', 'America/Indiana/Knox', 'America/Indiana/Vevay', 'America/Chicago', 'America/Indiana/Vincennes', 'America/Indiana/Petersburg', 'America/Menominee', 'America/North_Dakota/Center', 'America/North_Dakota/New_Salem', 'America/Denver', 'America/Boise', 'America/Shiprock', 'America/Phoenix', 'America/Los_Angeles', 'America/Anchorage', 'America/Juneau', 'America/Yakutat', 'America/Nome', 'America/Adak', 'Pacific/Honolulu'], 'code': 'US', 'continent': 'North America', 'name': 'United States', 'capital': 'Washington, D.C.'},
            {'timezones': ['America/Montevideo'], 'code': 'UY', 'continent': 'South America', 'name': 'Uruguay', 'capital': 'Montevideo'},
            {'timezones': ['Asia/Samarkand', 'Asia/Tashkent'], 'code': 'UZ', 'continent': 'Asia', 'name': 'Uzbekistan', 'capital': 'Tashkent'},
            {'timezones': ['Europe/Vatican'], 'code': 'VA', 'continent': 'Europe', 'name': 'Vatican City', 'capital': 'Vatican City'},
            {'timezones': ['America/Caracas'], 'code': 'VE', 'continent': 'South America', 'name': 'Venezuela', 'capital': 'Caracas'},
            {'timezones': ['Asia/Saigon'], 'code': 'VN', 'continent': 'Asia', 'name': 'Vietnam', 'capital': 'Hanoi'},
            {'timezones': ['Pacific/Efate'], 'code': 'VU', 'continent': 'Oceania', 'name': 'Vanuatu', 'capital': 'Port Vila'},
            {'timezones': ['Asia/Aden'], 'code': 'YE', 'continent': 'Asia', 'name': 'Yemen', 'capital': "Sana'a"},
            {'timezones': ['Africa/Lusaka'], 'code': 'ZM', 'continent': 'Africa', 'name': 'Zambia', 'capital': 'Lusaka'},
            {'timezones': ['Africa/Harare'], 'code': 'ZW', 'continent': 'Africa', 'name': 'Zimbabwe', 'capital': 'Harare'},
            {'timezones': ['Africa/Algiers'], 'code': 'DZ', 'continent': 'Africa', 'name': 'Algeria', 'capital': 'Algiers'},
            {'timezones': ['Europe/Sarajevo'], 'code': 'BA', 'continent': 'Europe', 'name': 'Bosnia and Herzegovina', 'capital': 'Sarajevo'},
            {'timezones': ['Asia/Phnom_Penh'], 'code': 'KH', 'continent': 'Asia', 'name': 'Cambodia', 'capital': 'Phnom Penh'},
            {'timezones': ['Africa/Bangui'], 'code': 'CF', 'continent': 'Africa', 'name': 'Central African Republic', 'capital': 'Bangui'},
            {'timezones': ['Africa/Ndjamena'], 'code': 'TD', 'continent': 'Africa', 'name': 'Chad', 'capital': "N'Djamena"},
            {'timezones': ['Indian/Comoro'], 'code': 'KM', 'continent': 'Africa', 'name': 'Comoros', 'capital': 'Moroni'},
            {'timezones': ['Europe/Zagreb'], 'code': 'HR', 'continent': 'Europe', 'name': 'Croatia', 'capital': 'Zagreb'},
            {'timezones': ['Asia/Dili'], 'code': 'TL', 'continent': 'Asia', 'name': 'East Timor', 'capital': 'Dili'},
            {'timezones': ['America/El_Salvador'], 'code': 'SV', 'continent': 'North America', 'name': 'El Salvador', 'capital': 'San Salvador'},
            {'timezones': ['Africa/Malabo'], 'code': 'GQ', 'continent': 'Africa', 'name': 'Equatorial Guinea', 'capital': 'Malabo'},
            {'timezones': ['America/Grenada'], 'code': 'GD', 'continent': 'North America', 'name': 'Grenada', 'capital': "St. George's"},
            {'timezones': ['Asia/Almaty', 'Asia/Qyzylorda', 'Asia/Aqtobe', 'Asia/Aqtau', 'Asia/Oral'], 'code': 'KZ', 'continent': 'Asia', 'name': 'Kazakhstan', 'capital': 'Astana'},
            {'timezones': ['Asia/Vientiane'], 'code': 'LA', 'continent': 'Asia', 'name': 'Laos', 'capital': 'Vientiane'},
            {'timezones': ['Pacific/Truk', 'Pacific/Ponape', 'Pacific/Kosrae'], 'code': 'FM', 'continent': 'Oceania', 'name': 'Federated States of Micronesia', 'capital': 'Palikir'},
            {'timezones': ['Europe/Chisinau'], 'code': 'MD', 'continent': 'Europe', 'name': 'Moldova', 'capital': 'Chi\xc5\x9fin\xc4\x83u'},
            {'timezones': ['Europe/Monaco'], 'code': 'MC', 'continent': 'Europe', 'name': 'Monaco', 'capital': 'Monaco'},
            {'timezones': ['Europe/Podgorica'], 'code': 'ME', 'continent': 'Europe', 'name': 'Montenegro', 'capital': 'Podgorica'},
            {'timezones': ['Africa/Casablanca'], 'code': 'MA', 'continent': 'Africa', 'name': 'Morocco', 'capital': 'Rabat'},
            {'timezones': ['America/St_Kitts'], 'code': 'KN', 'continent': 'North America', 'name': 'Saint Kitts and Nevis', 'capital': 'Basseterre'},
            {'timezones': ['America/St_Lucia'], 'code': 'LC', 'continent': 'North America', 'name': 'Saint Lucia', 'capital': 'Castries'},
            {'timezones': ['America/St_Vincent'], 'code': 'VC', 'continent': 'North America', 'name': 'Saint Vincent and the Grenadines', 'capital': 'Kingstown'},
            {'timezones': ['Pacific/Apia'], 'code': 'WS', 'continent': 'Oceania', 'name': 'Samoa', 'capital': 'Apia'},
            {'timezones': ['Europe/Belgrade'], 'code': 'RS', 'continent': 'Europe', 'name': 'Serbia', 'capital': 'Belgrade'},
            {'timezones': ['Africa/Johannesburg'], 'code': 'ZA', 'continent': 'Africa', 'name': 'South Africa', 'capital': 'Pretoria'},
            {'timezones': ['Europe/Madrid', 'Africa/Ceuta', 'Atlantic/Canary'], 'code': 'ES', 'continent': 'Europe', 'name': 'Spain', 'capital': 'Madrid'},
            {'timezones': ['Asia/Colombo'], 'code': 'LK', 'continent': 'Asia', 'name': 'Sri Lanka', 'capital': 'Sri Jayewardenepura Kotte'},
            {'timezones': ['Africa/Mbabane'], 'code': 'SZ', 'continent': 'Africa', 'name': 'Swaziland', 'capital': 'Mbabane'},
            {'timezones': ['Europe/Zurich'], 'code': 'CH', 'continent': 'Europe', 'name': 'Switzerland', 'capital': 'Bern'},
            {'timezones': ['Asia/Dubai'], 'code': 'AE', 'continent': 'Asia', 'name': 'United Arab Emirates', 'capital': 'Abu Dhabi'},
            {'timezones': ['Europe/London'], 'code': 'GB', 'continent': 'Europe', 'name': 'United Kingdom', 'capital': 'London'},
        ]
)

In [24]:
def country_to_continent(country):
    return country_continent["continent"][country_continent["name"] == country].tolist()[0]

jobs_by_country_NO_DATE = []

for country in relevant_countries:
    country_data = {
        "country": country,
        "iso_alpha3": world["iso_alpha3"][world["country"] == country].iloc[0],
        "continent": country_to_continent(country),
        "average_job_count": int(jobs_by_country[country]["count"].mean())
    }
    jobs_by_country_NO_DATE.append(country_data)

jobs_country_date_all = pd.DataFrame(jobs_by_country_NO_DATE)

In [25]:
fig = px.scatter_geo(jobs_country_date_all, locations="iso_alpha3", color="country",
                     hover_name="country", size="average_job_count", size_max=30,
                     projection="natural earth")
fig.show()

# END

#### Alot of conclusions can be drawn with our plots, and with more time we can even explore more. I'll probability come to it later after exploring a variety of other datasets.

## Please upvote if you liked this notebook, and comment your ideas/improvements!